# pandas vs polars

## TODO

- benchmark code:
   - aggregate

   - write

   - filter
   
   - join

   - group by


- docker container

- setup description (machine specs, docker, data manipulations)

- pandas, polars, mongoDB, apache spark (on a single node machine)

***

## Setup

In [34]:
# Installed packages
#pip install pandas
#pip install polars
#pip install scipy
#pip install matplotlib
#pip install pyarrow
#pip install fastparquet
#pip install seaborn
#pip install datetime

import time # measure runtime
import psutil # measure cpu usage and memory usage https://psutil.readthedocs.io/en/latest/
import numpy as np

import pandas as pd
print(f"pandas version used here is {pd.__version__}.")
import polars as pl
print(f"polars version used here is {pl.__version__}.")

import random
import string


# import datetime as dt

# from IPython.display import Image
# import seaborn as sns
# import matplotlib.pyplot as plt

# from scipy.stats import ttest_ind
# import statistics as st

pandas version used here is 2.1.1.
polars version used here is 0.19.7.


### Dataset

In [70]:

# This will be a eurpean CRM
european_countries = [
    'Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
    'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia',
    'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein',
    'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia',
    'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain',
    'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City'
]

num_rows = 10000000

# Generate CRM data
data = {
    'CustomerID': range(1, num_rows + 1),
    'Name': [''.join(random.choices(string.ascii_letters, k=5)) for _ in range(num_rows)],
    'Email': [''.join(random.choices(string.ascii_lowercase, k=7)) + '@example.com' for _ in range(num_rows)],
    'Phone': [''.join(random.choices(string.digits, k=10)) for _ in range(num_rows)],
    'Address': [''.join(random.choices(string.ascii_letters + string.digits, k=10)) for _ in range(num_rows)],
    'Country': [random.choice(european_countries) for _ in range(num_rows)],
    'Gender': [random.choice(['Male', 'Female']) for _ in range(num_rows)],
}

# Add random integer columns
for i in range(1, 11): # adjust range to increase amount of cols
    column_name = f'Int{i}'
    data[column_name] = random.choices(range(100), k=num_rows)

# Add random float columns
for i in range(1, 11): # adjust range to increase amount of cols
    column_name = f'Float{i}'
    data[column_name] = [random.uniform(0, 1) for _ in range(num_rows)]
    
df_pl = pl.DataFrame(data)
df_pd = pd.DataFrame(data)

### Function to track cpu & memory usage

Using psutils

In [5]:
def track_cpu_memory(query):
    cpu_usage_perc = []
    memory_usage_mb = []

    # start
    process = psutil.Process()
    start_time = time.time()
    start_cpu_percent = process.cpu_percent(interval=None)
    start_cpu_percent = process.cpu_percent(interval=None)
    start_memory_mb = process.memory_info().rss / (1024 * 1024)

    # run query
    result = query()

    # finish
    end_time = time.time()
    total_time = end_time - start_time
    end_cpu_percent = process.cpu_percent(interval=None)
    end_memory_mb = process.memory_info().rss / (1024 * 1024)

    # calculate cpu & memory
    cpu_usage_perc.append(end_cpu_percent - start_cpu_percent)
    memory_usage_mb.append(end_memory_mb - start_memory_mb)

    return {
        "avg_cpu_usage": cpu_usage_perc,
        "avg_memory_usage_mb": memory_usage_mb,
        "total_time": total_time
    }

Using psutils and pympler

In [ ]:
from pympler import asizeof

def track_cpu_memory_pympler(query):
    cpu_usage_perc = []
    memory_usage_mb = []

    # start
    process = psutil.Process()
    start_time = time.time()
    start_cpu_percent = process.cpu_percent(interval=None)
    start_memory_size = asizeof.asizeof(process.memory_info())

    # run query
    result = query()

    # finish
    end_time = time.time()
    total_time = end_time - start_time
    end_cpu_percent = process.cpu_percent(interval=None)
    end_memory_size = asizeof.asizeof(process.memory_info())

    # calculate cpu & memory
    cpu_usage_perc.append(end_cpu_percent - start_cpu_percent)
    memory_usage_mb.append((end_memory_size - start_memory_size) / (1024 * 1024))

    return {
        "avg_cpu_usage": cpu_usage_perc,
        "avg_memory_usage_mb": memory_usage_mb,
        "total_time": total_time
    }

Using memory profiler

In [74]:
from memory_profiler import profile
import random
import string
import pandas as pd
import polars as pl

@profile
def generate_random_data(num_rows=1000, num_int_cols=10, num_float_cols=10):
    european_countries = [
        'Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
        'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia',
        'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein',
        'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia',
        'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain',
        'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City'
    ]

    data = {
        'CustomerID': list(range(1, num_rows + 1)),
        'Name': [''.join(random.choices(string.ascii_letters, k=5)) for _ in range(num_rows)],
        'Email': [''.join(random.choices(string.ascii_lowercase, k=7)) + '@example.com' for _ in range(num_rows)],
        'Phone': [''.join(random.choices(string.digits, k=10)) for _ in range(num_rows)],
        'Address': [''.join(random.choices(string.ascii_letters + string.digits, k=10)) for _ in range(num_rows)],
        'Country': [random.choice(european_countries) for _ in range(num_rows)],
        'Gender': [random.choice(['Male', 'Female']) for _ in range(num_rows)]
    }

    data.update({f'Int{i}': [random.choices(range(100), k=num_rows) for _ in range(num_rows)] for i in range(1, num_int_cols + 1)})
    data.update({f'Float{i}': [random.uniform(0, 1) for _ in range(num_rows)] for i in range(1, num_float_cols + 1)})

    return data

# Generate CRM data
if __name__ == "__main__":
    data = generate_random_data()
    df_pl = pl.DataFrame(data)
    df_pd = pd.DataFrame(data)

# python -m memory_profiler myscript

#### Example query

In [6]:
def example_query():
    result = df_pd.max()
    return result

results = track_cpu_memory(example_query)

print(f"Average cpu usage: {results['avg_cpu_usage']}%")
print(f"Average memory usage: {results['avg_memory_usage_mb']} MB")
print(f"Total duration: {results['total_time']} seconds")

Average cpu usage: [26.1]%
Average memory usage: [48.65625] MB
Total duration: 0.3587985038757324 seconds


Print results function

In [63]:
def print_res(res_pd, res_pl):
    num_cores = psutil.cpu_count()
    print(f"Number of cpu cores: {num_cores}")
    print(f"Average cpu usage: \n\t pandas: {res_pd['avg_cpu_usage']}% \n\t polars: {res_pl['avg_cpu_usage']}% ({res_pl['avg_cpu_usage'][0]/num_cores:.2f}%/core)")
    print(f"Average memory usage: \n\t pandas: {res_pd['avg_memory_usage_mb']} MB \n\t polars: {res_pl['avg_memory_usage_mb']} MB")
    print(f"Total duration: \n\t pandas: {res_pd['total_time']} seconds \n\t polars: {res_pl['total_time']} seconds")

## Benchmarks and results
Queries/manipulations for benchmarking polars and pandas in terms of resource usage.



### Writing parquet-files

In [8]:
import os
os.getcwd()

'C:\\Users\\IvanEsin\\AppData\\Local\\Programs\\Microsoft VS Code'

In [25]:
def query_pd():
    pd_write=df_pd.to_parquet('test.parquet')
    pd_read=pd.read_parquet('./test.parquet')

res = track_cpu_memory(query_pd)

def query_pl():
    pl_write=df_pl.write_parquet('test.parquet')
    pl_read=pl.read_parquet('./test.parquet')

res = track_cpu_memory(query_pl)

print_res(res_pd, res_pl)

# print(f"Average cpu usage: {results['avg_cpu_usage']}%")
# print(f"Average memory usage: {results['avg_memory_usage_mb']} MB")
# print(f"Total duration: {results['total_time']} seconds")

Average cpu usage: 
	 pandas: [40.1]% 
	 polars: [299.2]%
Average memory usage: 
	 pandas: [149.07421875] MB 
	 polars: [36.07421875] MB
Total duration: 
	 pandas: 0.16077327728271484 seconds 
	 polars: 0.13962340354919434 seconds


### Aggregating

### Filtering

In [22]:
df_pd.columns

Index(['CustomerID', 'Name', 'Email', 'Phone', 'Address', 'Country', 'Gender',
       'Int1', 'Int2', 'Int3', 'Int4', 'Int5', 'Int6', 'Int7', 'Int8', 'Int9',
       'Int10', 'Float1', 'Float2', 'Float3', 'Float4', 'Float5', 'Float6',
       'Float7', 'Float8', 'Float9', 'Float10'],
      dtype='object')

**Query 0: filtering one integer value**

In [20]:
def query_pd():
    res = df_pd[df_pd['Int1'] >= 30]
    return res

def query_pl():
    res = df_pl.filter(pl.col("Int1") >= 30)
    return df_pl

res_pd = track_cpu_memory(query_pd)
res_pl = track_cpu_memory(query_pl)

print_res(res_pd, res_pl)

# print(f"Average cpu usage: \n\t pandas: {res_pd['avg_cpu_usage']}% \n\t polars: {res_pl['avg_cpu_usage']}%")
# print(f"Average memory usage: \n\t pandas: {res_pd['avg_memory_usage_mb']} MB \n\t polars: {res_pl['avg_memory_usage_mb']} MB")
# print(f"Total duration: \n\t pandas: {res_pd['total_time']} seconds \n\t polars: {res_pl['total_time']} seconds")

Average cpu usage: 
	 pandas: [40.1]% 
	 polars: [299.2]%
Average memory usage: 
	 pandas: [149.07421875] MB 
	 polars: [36.07421875] MB
Total duration: 
	 pandas: 0.16077327728271484 seconds 
	 polars: 0.13962340354919434 seconds


**Query 1: filter and select as in https://medium.com/cuenex/pandas-2-0-vs-polars-the-ultimate-battle-a378eb75d6d1**

Using track_cpu_memory()

In [65]:
def query_pd():
    res = df_pd[df_pd['Int1']<=10][['Country', 'Gender']].nunique()
    return res

def query_pl():
    res = df_pl.filter(pl.col("Int1") <= 10).select(pl.col(['Country', 'Gender']).n_unique())
    return res

res_pd = track_cpu_memory(query_pd)
res_pl = track_cpu_memory(query_pl)

print_res(res_pd, res_pl)


Number of cpu cores: 12
Average cpu usage: 
	 pandas: [40.1]% 
	 polars: [243.7]% (20.31%/core)
Average memory usage: 
	 pandas: [0.0] MB 
	 polars: [1.0625] MB
Total duration: 
	 pandas: 0.08094239234924316 seconds 
	 polars: 0.10825657844543457 seconds


Using track_cpu_memory_pympler()

In [72]:
def query_pd():
    res = df_pd[df_pd['Int1']<=10][['Country', 'Gender']].nunique()
    return res

def query_pl():
    res = df_pl.filter(pl.col("Int1") <= 10).select(pl.col(['Country', 'Gender']).n_unique())
    return res

res_pd = track_cpu_memory_pympler(query_pd)
res_pl = track_cpu_memory_pympler(query_pl)

print_res(res_pd, res_pl)

Number of cpu cores: 12
Average cpu usage: 
	 pandas: [6.2]% 
	 polars: [176.1]% (14.67%/core)
Average memory usage: 
	 pandas: [0.0] MB 
	 polars: [0.0] MB
Total duration: 
	 pandas: 15.230718612670898 seconds 
	 polars: 4.326357841491699 seconds


**Query 2: the mean of all numerical columns when cat_1 equals 1**

Using track_cpu_memory()

In [71]:
def query_pd():
    res = df_pd[df_pd['Int1']<=10][['Country', 'Gender']].nunique()
    res = df_pd[df_pd['Int1']<=10][['Int2', 'Int3', 'Float1', 'Float2', 'Float3', 'Float4']].mean()
    return res

def query_pl():
    res = df_pl.filter(pl.col("Country") == 'Belgium').select(pl.col(['Int2', 'Int3', 'Float1', 'Float2', 'Float3', 'Float4']).mean())
    return res

res_pd = track_cpu_memory(query_pd)
res_pl = track_cpu_memory(query_pl)

print_res(res_pd, res_pl)

Number of cpu cores: 12
Average cpu usage: 
	 pandas: [11.0]% 
	 polars: [37.5]% (3.12%/core)
Average memory usage: 
	 pandas: [4347.21484375] MB 
	 polars: [-3947.97265625] MB
Total duration: 
	 pandas: 21.441218614578247 seconds 
	 polars: 4.430878400802612 seconds


Using track_cpu_memory_pympler()

In [73]:
def query_pd():
    res = df_pd[df_pd['Int1']<=10][['Country', 'Gender']].nunique()
    res = df_pd[df_pd['Int1']<=10][['Int2', 'Int3', 'Float1', 'Float2', 'Float3', 'Float4']].mean()
    return res

def query_pl():
    res = df_pl.filter(pl.col("Country") == 'Belgium').select(pl.col(['Int2', 'Int3', 'Float1', 'Float2', 'Float3', 'Float4']).mean())
    return res

res_pd = track_cpu_memory_pympler(query_pd)
res_pl = track_cpu_memory_pympler(query_pl)

print_res(res_pd, res_pl)

Number of cpu cores: 12
Average cpu usage: 
	 pandas: [6.3]% 
	 polars: [149.8]% (12.48%/core)
Average memory usage: 
	 pandas: [0.0] MB 
	 polars: [0.0] MB
Total duration: 
	 pandas: 21.69314193725586 seconds 
	 polars: 4.1940083503723145 seconds


### Joining

### Grouping

## Discussion